<h2 align='center'>ML Flow Tutorial</h2>

In [13]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [15]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [16]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [17]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [18]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [19]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [20]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/06/26 15:24:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/26 15:24:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/26 15:24:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/846313948183712985/runs/b90654f54c2d49139cf30750c766654b
🧪 View experiment at: http://localhost:5000/#/experiments/846313948183712985


2025/06/26 15:24:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/26 15:24:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/846313948183712985/runs/7986df0a779d470eb932c20dfa850c39
🧪 View experiment at: http://localhost:5000/#/experiments/846313948183712985


2025/06/26 15:24:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/26 15:24:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/846313948183712985/runs/1d0f72d622d7478a895bb38be44871fc
🧪 View experiment at: http://localhost:5000/#/experiments/846313948183712985


2025/06/26 15:24:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/846313948183712985/runs/54ece498ac9a442cb6c17cb0492b28fc
🧪 View experiment at: http://localhost:5000/#/experiments/846313948183712985


### Register the Model

In [27]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri, name=model_name)

if mlflow.active_run():
    mlflow.end_run()
    
with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 54ece498ac9a442cb6c17cb0492b28fc


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/06/26 15:34:18 WARNING mlflow.tracking._model_registry.fluent: Run with id 54ece498ac9a442cb6c17cb0492b28fc has no artifacts at artifact path 'model', registering model based on models:/m-e68d8c77b74e4d248871d3c929b948c4 instead
2025/06/26 15:34:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 3
Created version '3' of model 'XGB-Smote'.
Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/06/26 15:34:18 WARNING mlflow.tracking._model_registry.fluent: Run with id 54ece498ac9a442cb6c17cb0492b28fc has no artifacts at artifact path 'model', registering model based on models:/m-e68d8c77b74e4d248871d3c929b948c4 instead
2025/06/26 15:34:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote

🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/846313948183712985/runs/54ece498ac9a442cb6c17cb0492b28fc
🧪 View experiment at: http://localhost:5000/#/experiments/846313948183712985


Created version '4' of model 'XGB-Smote'.


### Load the Model

In [28]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [32]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

def get_version_by_tag(model_name, tag_key, tag_value):
    """Return the version of a model that matches a given tag."""
    for mv in client.search_model_versions(f"name='{model_name}'"):
        tags = dict(mv.tags)
        if tags.get(tag_key) == tag_value:
            return mv.version
    return None


### Transition the Model to Production

In [33]:
# Define your source and destination model names
model_name = "XGB-Smote"
tag_key = "stage"
tag_value = "challenger"
production_model_name = "anomaly-detection-prod"

# Get version number using tag
version = get_version_by_tag(model_name, tag_key, tag_value)

if version:
    current_model_uri = f"models:/{model_name}/{version}"
    client.copy_model_version(
        src_model_uri=current_model_uri,
        dst_name=production_model_name
    )
    print(f"✅ Copied version {version} of {model_name} to {production_model_name}")
else:
    print(f"❌ No version of {model_name} with tag '{tag_key}={tag_value}' found.")

Successfully registered model 'anomaly-detection-prod'.


✅ Copied version 4 of XGB-Smote to anomaly-detection-prod


Copied version '4' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


In [38]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 